# Indeed Job Scraper
create a general purpose job scraper

In [1]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

### getting jobs from indeed with informations with just giving a job title 

In [17]:
import csv
from datetime import datetime
import requests
import re
from bs4 import BeautifulSoup


def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://fr.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url


def get_record(card):
    """Extract job data from a single record"""
    
    job_title = card.h2.span.get('title')
    company = card.find('span', 'companyName').text
    job_location = card.find('div', 'companyLocation').text
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'job-snippet').find_all('li')
    summary = summary[0].get_text()
    #job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''  
        
    record = (job_title, company, job_location, post_date, today, summary, salary)#, job_url)
    return record


def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)
    
    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break
    print(records)
        
    # save the job data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Salary'])
        writer.writerows(records)

In [18]:
# run the main program
main('data engineer', 'paris')

[(None, 'Siderlog', 'Paris (75)', 'il y a 7 jours', '2021-11-03', 'Vous travaillez en relation directe avec le client et intégrez une équipe de consultants Siderlog.', ''), ('Data Engineer Junior', 'NonStop Consulting', 'Clichy (92)', 'il y a 12 jours', '2021-11-03', "L'entreprise en question est une start-up en plein développement qui recherche Data Engineer Junior en Freelance ou en CDI à CLICHY (92) en région parisienne.", ''), (None, 'Capgemini Invent', 'Paris (75)', 'il y a 3 jours', '2021-11-03', 'Sectoriel Data Strategy est une entité multidisciplinaire regroupant consultants digitaux, data scientists, data engineers ainsi que des profils technologiques …', ''), (None, 'Alstom', 'Saint-Ouen (93)', 'il y a 1 jour', '2021-11-03', 'Take in charge the correction of the data and their segmentation according to the protocols as defined.', ''), (None, 'Société Générale', 'La Défense (92)+ 1\xa0lieu', 'il y a 8 jours', '2021-11-03', 'Le Département Digital Transformation Office de la Di